<a href="https://colab.research.google.com/github/AryaPratamaPutra-10/Analyst-Uang/blob/main/Selamat_Datang_di_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aryapratamaputra10","key":"95340623c73554e9a82c2177872296e0"}'}

In [2]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
!kaggle datasets download
import os
import zipfile

# Pastikan Kaggle API Key sudah terkonfigurasi
!mkdir -p ~/.kaggle
!cp /path/to/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Tentukan dataset Kaggle dan tujuan
dataset_name = "yunhann/indonesian-rupiah-emission-2022-real-and-fake"
output_dir = "/content/rupiah-real-fake-dataset"

# Unduh dataset
!kaggle datasets download -d {dataset_name} -p /content

# Cari file ZIP yang sesuai
zip_file = f"/content/{dataset_name.split('/')[-1]}.zip"

# Ekstrak file ZIP ke folder tujuan
if os.path.exists(zip_file):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    print(f"Dataset berhasil diekstrak ke: {output_dir}")

    # Hapus file ZIP untuk menghemat ruang
    os.remove(zip_file)
else:
    print("File ZIP tidak ditemukan!")


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 63, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1569, in dataset_download_cli
    owner_slug, dataset_slug, _ = self.split_dataset_string(dataset)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 4162, in split_dataset_string
    if '/' in dataset:
       ^^^^^^^^^^^^^^
TypeError: argument of type 'NoneType' is not iterable
cp: cannot stat '/path/to/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/yunhann/indonesian-rupiah-emission-2022-real-and-fake
L

Labelling Data

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = "/content/rupiah-real-fake-dataset"


# Buat objek ImageDataGenerator untuk preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalisasi pixel (0-1)
    validation_split=0.1  # 10% data untuk validasi
)


train_generator = datagen.flow_from_directory(
    "/content/rupiah-real-fake-dataset/dataset",  # Arahkan langsung ke "dataset"
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    "/content/rupiah-real-fake-dataset/dataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)




train_generator = datagen.flow_from_directory(
    "/content/rupiah-real-fake-dataset/dataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Multi-class classification
)

label_mapping = {
    "uang 100 asli": 0,
    "uang 100 palsu": 1,
    "uang 50 asli": 2,
    "uang 50 palsu": 3
}


print("Classes:", train_generator.class_indices)


Found 198 images belonging to 4 classes.
Found 22 images belonging to 4 classes.
Found 220 images belonging to 4 classes.
Classes: {'uang100asli': 0, 'uang100palsu': 1, 'uang50asli': 2, 'uang50palsu': 3}


Make a Model with CNN